<a href="https://colab.research.google.com/github/aninsung/Machine-Learning-Programming/blob/main/4%EC%A3%BC%EC%B0%A8_mhealth_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (오늘 실습과제)

Mhealth dataset으로 인간행동 분류하기

일단 데이터 읽어서 subject 번호가 컬럼면에 포함된 통합 data frame 만들고 CNN, LSTM, CNN-LSTM 해보기.

1. LLM(sLLM)을 제외한 어떤 모델을 써도 가능. 단 라이브러리 import 불가. 모델의 풀 소스가 있어야 함.

 2. 10명을 1+2, 3+4, 4+5,.... 5개그룹으로 나눠 2명씩 테스트 데이터로 하면서 cross validation.

 3. 데이터 증대는 안됨.

4. 오직 모델로만 성능 향상

5. 상위 30%씩 차등.

6. 성적에 반영

7. 11월15일까지. Github게시

교차 평가로 평균 F1 기준.

8. 본인이 완벽하게 소스 및 모델을 설명할 수 있어야 함. GPT를 쓰던 뭘 쓰던 그건 상관안함. 재현성이 보장되어야 함.
--------------------------------------------------------------------------
--------------------------------------------------------------------------

## 데이터설명
📌 Dataset Summary

Subjects: 10명 (다양한 프로필의 자원자)

Activities: 12개

Sensors: 3개 (가슴, 오른쪽 손목, 왼쪽 발목 → elastic strap으로 고정)

측정 정보:

Acceleration (가속도)

Gyroscope (회전율)

Magnetometer (자기장 방향)

Chest sensor: 2-lead ECG 추가 (활동 인식에는 사용 안 함, 향후 연구용)

Sampling Rate: 50 Hz

환경: Out-of-lab, 제약 없이 자유롭게 활동 수행 (단, 최선을 다해 실행)

📌 Activity Set (12 classes)

정적 활동: Standing still, Sitting, Lying down

일상/이동 활동: Walking, Climbing stairs, Cycling

운동/역동적 활동: Jogging, Running, Jump front & back

반복 동작: Waist bends, Arm elevation, Knees bending

📌 특징

ECG 포함: 운동 중 심전도 기록 → 부정맥, 심박수 모니터링, 운동 효과 분석 가능

다양성 확보:

동작 강도 차이 (예: Sitting vs. Running)

속도 차이 (예: Walking vs. Jogging)

다른 신체 부위 동작 (예: Arm elevation vs. Knees bending)

실생활 반영: 실제 환경에서 수행되어 일반화 성능 높음

In [52]:
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, Conv1D, MaxPooling1D, TimeDistributed
from tensorflow.keras.utils import to_categorical
import os

# 시각화 스타일 설정
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

In [53]:
DATASET_BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/mhealth+dataset/MHEALTHDATASET"


# 10개 주제의 모든 데이터를 로드
all_dfs = []
column_names = [
    'chest_acc_x', 'chest_acc_y', 'chest_acc_z', 'ecg_lead1', 'ecg_lead2', 'left_ankle_acc_x',
    'left_ankle_acc_y', 'left_ankle_acc_z', 'left_ankle_gyro_x', 'left_ankle_gyro_y',
    'left_ankle_gyro_z', 'left_ankle_mag_x', 'left_ankle_mag_y', 'left_ankle_mag_z',
    'right_wrist_acc_x', 'right_wrist_acc_y', 'right_wrist_acc_z', 'right_wrist_gyro_x',
    'right_wrist_gyro_y', 'right_wrist_gyro_z', 'right_wrist_mag_x', 'right_wrist_mag_y',
    'right_wrist_mag_z', 'activity_label'
]

for i in range(1, 11):
    file_path = os.path.join(DATASET_BASE_PATH, f"mHealth_subject{i}.log")
    try:
        df_subject = pd.read_csv(file_path, sep='\s+', header=None, names=column_names)
        df_subject['subject'] = i  # **<-- 이 라인이 subject 컬럼을 생성합니다**
        all_dfs.append(df_subject)
    except FileNotFoundError:
        all_dfs = []
        break

if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)

    # Activity 0 (Null) 제거 및 레이블 인코딩
    df = df[df['activity_label'] != 0].copy()
    le = LabelEncoder()
    df['activity_label'] = le.fit_transform(df['activity_label'])

    print("데이터 로드 및 정제 완료.")

<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-3428356089.py:18: SyntaxWarning: invalid escape sequence '\s'
  df_subject = pd.read_csv(file_path, sep='\s+', header=None, names=column_names)


데이터 로드 및 정제 완료.


In [54]:
# 훈련 데이터를 기준으로 스케일러 학습 및 적용
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("데이터 스케일링 완료.")

데이터 스케일링 완료.


In [55]:
# Reformat the training data into sequence samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [56]:
prepared_data = []
n_steps = 25

if 'df' in locals():
    # **<-- 'activity'가 아닌 'activity_label'을 사용하도록 수정**
    feature_cols = df.columns.drop(['activity_label', 'subject'])
    label_col = 'activity_label'
    num_classes = df[label_col].nunique()

    test_subject_pairs = [(i, i + 1) for i in range(1, 11, 2)]

    for test_subjects in test_subject_pairs:
        train_df = df[~df['subject'].isin(test_subjects)].copy()
        test_df = df[df['subject'].isin(test_subjects)].copy()

        scaler = StandardScaler()
        train_df.loc[:, feature_cols] = scaler.fit_transform(train_df[feature_cols])
        test_df.loc[:, feature_cols] = scaler.transform(test_df[feature_cols])

        y_train_array = np.array(train_df[label_col])
        train_set = np.c_[train_df[feature_cols], y_train_array]

        y_test_array = np.array(test_df[label_col])
        test_set = np.c_[test_df[feature_cols], y_test_array]

        X_train, y_train = split_sequences(train_set, n_steps)
        X_test, y_test = split_sequences(test_set, n_steps)

        prepared_data.append({'X_train': X_train, 'y_train': y_train, 'X_test': X_test, 'y_test': y_test})

    print(f"총 {len(prepared_data)}개의 교차검증 폴드 데이터 준비 완료.")

총 5개의 교차검증 폴드 데이터 준비 완료.


In [57]:
n_timesteps, n_features, n_outputs = X_train_seq.shape[1], X_train_seq.shape[2], y_train_cat.shape[1]

# 모델 정의
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(n_outputs, activation='softmax')
])
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("--- CNN 모델 구조 ---")
cnn_model.summary()


--- CNN 모델 구조 ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 23, 64)         │         4,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 704)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        70,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 13)             │         1,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 76,293 (298.02 KB)

 Trainable params: 76,293 (298.02 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:

# 모델 훈련
cnn_history = cnn_model.fit(X_train_seq, y_train_cat, epochs=10, batch_size=64, validation_data=(X_test_seq, y_test_cat), verbose=1)

Epoch 1/10
14247/14247 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.7176 - loss: 1.3046 - val_accuracy: 0.7179 - val_loss: 1.2929
Epoch 2/10
14247/14247 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - accuracy: 0.7173 - loss: 1.2945 - val_accuracy: 0.7179 - val_loss: 1.2907
Epoch 3/10
14247/14247 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - accuracy: 0.7177 - loss: 1.2920 - val_accuracy: 0.7179 - val_loss: 1.2909
Epoch 4/10
14247/14247 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - accuracy: 0.7174 - loss: 1.2922 - val_accuracy: 0.7179 - val_loss: 1.2913
Epoch 5/10
14247/14247 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - accuracy: 0.7174 - loss: 1.2914 - val_accuracy: 0.7179 - val_loss: 1.2919
Epoch 6/10
14247/14247 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - accuracy: 0.7180 - loss: 1.2891 - val_accuracy: 0.7179 - val_loss: 1.2908
Epoch 7/10
14247/14247 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - accuracy: 0.7176 - loss: 1.2900 - val_accuracy: 0.7179 - val_loss: 1.2899
Epoch 8/10
14247/14247 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - accuracy: 

In [ ]:
# 모델 정의
lstm_model = Sequential([
    LSTM(units=128, input_shape=(n_timesteps, n_features)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(n_outputs, activation='softmax')
])
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("--- LSTM 모델 구조 ---")
lstm_model.summary()

# 모델 훈련
lstm_history = lstm_model.fit(X_train_seq, y_train_cat, epochs=10, batch_size=64, validation_data=(X_test_seq, y_test_cat), verbose=1)

--- LSTM 모델 구조 ---


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 128)            │        77,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 13)             │         1,677 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 96,013 (375.05 KB)

 Trainable params: 96,013 (375.05 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
 7323/14247 ━━━━━━━━━━━━━━━━━━━━ 1:42 15ms/step - accuracy: 0.7505 - loss: 0.7829

In [ ]:
# CNN-LSTM 입력에 맞게 데이터 형태 변경 (subsequences=1 추가)
n_seq, n_steps, n_features = X_train_seq.shape
X_train_reshaped = X_train_seq.reshape((n_seq, 1, n_steps, n_features))
X_test_reshaped = X_test_seq.reshape((X_test_seq.shape[0], 1, n_steps, n_features))

# 모델 정의
cnn_lstm_model = Sequential([
    TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None, n_steps, n_features)),
    TimeDistributed(MaxPooling1D(pool_size=2)),
    TimeDistributed(Flatten()),
    LSTM(100),
    Dense(100, activation='relu'),
    Dense(n_outputs, activation='softmax')
])
cnn_lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("--- CNN-LSTM 모델 구조 ---")
cnn_lstm_model.summary()

# 모델 훈련
cnn_lstm_history = cnn_lstm_model.fit(X_train_reshaped, y_train_cat, epochs=10, batch_size=64, validation_data=(X_test_reshaped, y_test_cat), verbose=1)

In [ ]:
   plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=activity_map.values(),
                yticklabels=activity_map.values())
    plt.title('Confusion Matrix for CNN-LSTM (Last Fold)')
    plt.ylabel('Actual Activity')
    plt.xlabel('Predicted Activity')
    plt.show()

In [ ]:
# 평가 결과 요약 함수 정의
def results_summarizer(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)

    activity_map = {
        0: 'Standing still', 1: 'Sitting', 2: 'Lying down',
        3: 'Walking', 4: 'Climbing stairs', 5: 'Waist bends',
        6: 'Arm elevation', 7: 'Knees bending', 8: 'Cycling',
        9: 'Jogging', 10: 'Running', 11: 'Jumping'
    }

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=activity_map.values(),
                yticklabels=activity_map.values())
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Activity')
    plt.ylabel('Actual Activity')
    plt.show()

    print(f'Accuracy Score: ' + '{:.4%}'.format(acc))
    print("\n--- Classification Report ---")
    print(classification_report(y_true, y_pred, target_names=activity_map.values()))

# 마지막으로 훈련된 CNN-LSTM 모델의 예측 수행
y_pred_one_hot = cnn_lstm_model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_one_hot, axis=1)

# 평가 결과 출력
print("===== 최종 모델(CNN-LSTM) 평가 결과 =====")
results_summarizer(y_test_seq, y_pred)